In [0]:
%tensorflow_version 1.x

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

Restart RUNTIME after selected version of tensorflow

In [0]:
import os
os.kill(os.getpid(), 9)

In [0]:
import tensorflow as tf
print(tf.__version__) 

1.15.0


Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


cd ML

In [0]:
%cd '/content/drive/My Drive/SMART_SCOOTER/'

**Clone the Tensorflow models repository and the BOSCH github repository (JUST 1 TIME). Also you may clone the COCO repository and install the COCO object detection API for evaluation purpose**

In [0]:
!git clone https://github.com/tensorflow/models.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

fatal: destination path 'models' already exists and is not an empty directory.
/bin/bash: line 0: cd: cocoapi/PythonAPI: No such file or directory
make: *** No targets specified and no makefile found.  Stop.
cp: cannot stat 'pycocotools': No such file or directory


In [0]:
!git clone https://github.com/bosch-ros-pkg/bstld.git

fatal: destination path 'bstld' already exists and is not an empty directory.


**NEEDED TO RUN EVERY NEW RUNTIME**


In [2]:
%cd '/content/drive/My Drive/SMART_SCOOTER/'
#1. Install some needed tools
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#2. Compile the model definitionn
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/
!protoc object_detection/protos/*.proto --python_out=.
#3. Set the environment
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/SMART_SCOOTER/models/research/:/content/drive/My Drive/SMART_SCOOTER/models/research/slim'
#4. Always run this every restart of session
!python setup.py build
!python setup.py install
#5. Test to see if all we need for the training has been installed
!python object_detection/builders/model_builder_test.py

/content/drive/My Drive/SMART_SCOOTER
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://arc

In [0]:
#unzip dataset
%cd '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images'
!7za x dataset_test_rgb.zip.001
!7za x dataset_test_rgb.zip.002
!7za x dataset_train_rgb.zip.001
!7za x dataset_train_rgb.zip.002
!7za x dataset_train_rgb.zip.003

**PREPARATION ENDED**


---




**NEXT:**

We need to create a tensorflow record file from the xml file we have. For this we will be using some of Dat Tran’s code for conversion of XML_TO CSV (modified) and to generate TFRECORD (modified) doing a little correction to suit my need.




**STEPS to create CSV ANNOTATIONS FILES:**


1.   Save image folder (RGB) with: test and train folders with the images in ../IMAGES/TL_V#
2.   Run the bosch_to_pascal.py file from: https://github.com/bosch-ros-pkg/bstld from where we will create the xml files: 
`!python bosch_to_pascal.py input_yaml out_folder`


Modification in bosch_to_pascal.py to save the img path in filename
```
    ET.SubElement(annotation, 'filename').text = str(impath.split('./')[1])
```



> I have modified the bosch_to_pascal.py to just create those xml files of the images we have imported in /images/



> Make sure to import the annotations of the images you have downloaded from the bosch dataset (train/test.yaml)



3. Edit the PATHS of xml_to_csv to search correctly the xml files (look inside the file to make sure the path is correct)





In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/bstld/
!python bosch_to_pascal.py '/content/drive/My Drive/SMART_SCOOTER/bstld/label_files/train.yaml' '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/train_xml_annotations/'
!python bosch_to_pascal.py '/content/drive/My Drive/SMART_SCOOTER/bstld/label_files/test.yaml' '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/test_xml_annotations/'

Change image_path in xml_to_csv (TL_V1 in this case): 

```
def main():
  for folder in ['train', 'test']:
        image_path = os.path.join(os.getcwd(), ('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/'+folder+'_xml_annotations/' ))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv(('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/'+folder+'_csv_annotations/'+folder+'_labels.csv'), index=None)
  print('Successfully converted xml to csv.')


main()
```
  



In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/raccoon_dataset
!python xml_to_csv.py

Export the xml_to_csv.py and generate_tfrecord.py from: https://github.com/datitran/raccoon_dataset

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/
!git clone https://github.com/datitran/raccoon_dataset.git

**STEPS to create the TRAIN/TEST.records correctly**

1. Make sure the .csv has been correctly created ([filename,h,w,class,xm,ym...])

2. Compare our tf generator with the one from:https://github.com/bosch-ros-pkg/bstld/blob/master/tf_object_detection/to_tfrecords.py CHANGE THE PATHS!!!

---
The .records are files that collects all those images labeled (empty or not) in an 'array' with: filename (and its folder), height, weigth, class, and position. FOR EVERY IMAGE LABELED ONLY.

The .record's are the only files that contains information of the dataset for the training!!!!


Let's create train.record and test.record with generate_tfrecord.py.

we have to create a new file for a new and diferent training.



```
def class_text_to_int(row_label):
    if row_label == 'Off':
        return 1
    elif row_label == 'Green':
        return 2
    elif row_label == 'Yellow':
        return 3
    elif row_label == 'Red':
        return 4
    else:
        return 0
```

There has been also modifications in the variables to write in .record, to define png format and encode 'utf8'

In [0]:
%cd '/content/drive/My Drive/SMART_SCOOTER/raccoon_dataset'

!python generate_tfrecord.py --csv_input='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/train_csv_annotations/train_labels.csv' --image_dir='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/' --output_path='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/train_record/train.record'
!python generate_tfrecord.py --csv_input='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/test_csv_annotations/test_labels.csv' --image_dir='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/' --output_path='/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/test_record/test.record'

Get the pre-trained Object detection model from tensorflow with the code below

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/models
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!tar -xvf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz 

We will save the config in
object_detection/training/TL_V#

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/

In [0]:
%cp '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/samples/configs/ssdlite_mobilenet_v2_coco.config' training/TL_V1/

You should also modify the checkpoint path, num_classes, num_examples, and label_map_path.

A label map file called {model}.pbtxt must be localizwed and saved in both ‘data’, and ‘training’ folders. the label map will look like below code!!!!



```
item {
  id: 1
  name: 'off'
}

item {
  id: 2
  name: 'Green'
}

item {
  id: 3
  name: 'Yellow'
}

item {
  id: 4
  name: 'Red'
}
```



In [0]:
%cp '/content/drive/My Drive/SMART_SCOOTER/bstld/tf_object_detection/label_maps/bstld_label_map.pbtxt' '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1'
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1/
%mv bstld_label_map.pbtxt TL_V1.pbtxt 

Also get the config file which you might need to edit it can be found in object_detection/samples/configs. Please note the directories

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1/
%mv ssdlite_mobilenet_v2_coco.config TL_V1.config

**Remember to edit: TL_v1.config**

1. num_classes=4

2. set anchor boxes aspect ratios

3. fine_tune_checkpoint: "/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt"
> Fine_tune_checkpoint: "/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/new_models/TL_V1/model.ckpt-XXXX"

3. set batch_size=24/16/8, V1->8

5. num_steps=1000

4. input_path: "/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/train_record/train.record"

5. input_path: "/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/test_record/test.record"

6. label_map_path: "/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1/TL_V#.pbtxt"

7. 

```
image_resizer {
      fixed_shape_resizer {
        height: 720
        width: 1280
      }
```
8. num_examples: 1438 (test images)






---


**The TRAINING PROPER!**

Open your google drive tab and go to Legacy folder in the object detection directory, copy or move the train.py file into the object detection folder. then go back to Colab and run the training with the code below. train_dir (train.py), model_dir (model_main.py)

In [0]:
%tensorboard --logdir /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1

In [0]:
%cp '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/legacy/train.py' '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/'

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
!python train.py --logtostderr --train_dir=training/TL_V1/ --pipeline_config_path=training/TL_V1/TL_V1.config

Export your inference graph

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection
!python export_inference_graph.py --input_type image_tensor  --pipeline_config_path training/TL_V1/TL_V1.config   --trained_checkpoint_prefix training/TL_V1/model.ckpt-15000   --output_directory new_models/TL_V1

Zip your inference saved on object_detection

In [0]:
!zip -r TL_V1.zip new_models/TL_V1

Create files to run the network on opencv. 
In other words, we need to create a graph.pbtxtt, but we already have it from the training itself

In [0]:
!wget https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_ssd.py
!wget https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_common.py

In [4]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
!python tf_text_graph_ssd.py --input new_models/TL_V1/frozen_inference_graph.pb --config training/TL_V1/TL_V1.config --output new_models/TL_V1/graph.pbtxt

/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection
Scale: [0.200000-0.950000]
Aspect ratios: [1.0, 2.0, 0.5, 3.0, 0.3333]
Reduce boxes in the lowest layer: True
Number of classes: 4
Number of layers: 6
box predictor: convolutional
Input image size: 300x300


TEST YOUR INFERENCE GRAPH


In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection


/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection


In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/utils
# This is needed since the notebook is stored in the object_detection folder.

sys.path.append("..")
import utils
#sys.path.append("..")
import label_map_util

import visualization_utils as vis_util

/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/utils


In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed to display the images.
%matplotlib inline


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/new_models/TL_V1/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1', 'TL_V1.pbtxt')

NUM_CLASSES = 4

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')



label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/rgb_jpg/train/2015-10-05-14-40-46_bag'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}'.format(i)) for i in os.listdir('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/rgb_jpg/train/2015-10-05-14-40-46_bag') ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
c=0
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
         if c<6: 
          if image_path.endswith('.jpg'):
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            c+=1

Output hidden; open in https://colab.research.google.com to view.